250.515

ts
Athletes['asher hong' 'yul moldauer' 'khoi young' 'fred richard']
 Score:
260.579

hh

 Athletes: ['javier alfonso' 'vitaliy guimaraes' 'fred richard' 'yul moldauer']
 Score: 254.318

vns
 Athletes: ['fuzzy benas', 'yul moldauer', 'asher hong', 'fred richard']
 Score: 257.34200000000004

In [22]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../../../Combine_Data/men/iaa2024.csv')


In [4]:
df=df[df['Nation']=='USA']

In [5]:
df=df.drop(columns=['Rank', 'Nation', 'round', 'year', 'AA'])

In [6]:
# Group by 'Athlete' and get the index of the maximum score for each apparatus
idx_max = df.groupby('Athlete').idxmax()

# Create a new DataFrame with the highest scores for each athlete
df_max_scores = pd.DataFrame({
    'Athlete': df.loc[idx_max['VT'], 'Athlete'].values,
    'VT': df.loc[idx_max['VT'], 'VT'].values,
    'PH': df.loc[idx_max['PH'], 'PH'].values,
    'SR': df.loc[idx_max['SR'], 'SR'].values,
    'FX': df.loc[idx_max['FX'], 'FX'].values,
    'PB': df.loc[idx_max['PB'], 'PB'].values,
    'HB': df.loc[idx_max['HB'], 'HB'].values
})


In [7]:
df = df_max_scores.copy()

In [8]:
import pandas as pd
import numpy as np
from itertools import combinations
from pulp import LpProblem, LpVariable, lpSum, LpMaximize


In [9]:
df['Athlete'].value_counts()

Athlete
alex diab                1
raydel gamboa            1
kazuki hayashi           1
khoi young               1
landen blixt             1
mathew davis             1
matt cormier             1
maxim bereznev           1
michael artlip           1
michael jaroh            1
mike fletcher            1
nicolas kuebler          1
noah newfeld             1
patrick hoopes           1
riley loos               1
anthony koppie           1
rithik puri              1
robert neff              1
shane wiskus             1
spencer goodell          1
stephen nedoroscik       1
taylor burkhart          1
taylor christopulos      1
toby liang               1
troy lipis               1
vahe petrosyan           1
vitaliy guimaraes        1
yul moldauer             1
kameron nelson           1
joshua karnes            1
jeremy bischoff          1
javier alfonso           1
asher hong               1
blake sun                1
brandon dang             1
caleb melton             1
cameron bock        

vns
Selected Athletes: ['fuzzy benas', 'yul moldauer', 'asher hong', 'fred richard']
Total Score: 257.34200000000004

In [10]:
# VNS Algorithm with Iteration through random_state values
def variable_neighborhood_search(df):
    apparatus_order = ['FX', 'PH', 'SR', 'VT', 'PB', 'HB']  

    def calculate_total_score(athletes_df):
        # Sum the top 3 scores for each apparatus
        total_score = athletes_df[['FX', 'PH', 'SR', 'VT', 'PB', 'HB']].apply(lambda x: x.nlargest(3).sum()).sum()
        return total_score

    best_athletes = None
    best_score = float('-inf')
    best_seed = None

    for seed in range(6000): 
        # Randomly select 4 athletes with the current seed
        selected_athletes = df.sample(4, random_state=seed)

        # Check if each apparatus is competed on 3 times
        if all(selected_athletes[apparatus].count() >= 3 for apparatus in apparatus_order):
            # Calculate the total score for the current solution
            current_score = calculate_total_score(selected_athletes)

            # Compare with the current best solution
            if current_score > best_score:
                best_athletes = selected_athletes['Athlete'].tolist()
                best_score = current_score
                best_seed = seed

    return best_athletes, best_score, best_seed

# Run the VNS algorithm with different random_state values
best_athletes, total_score, best_seed = variable_neighborhood_search(df)

print("Selected Athletes:", best_athletes)
print("Total Score:", total_score)
print("Best Seed:", best_seed)


Selected Athletes: ['fuzzy benas', 'yul moldauer', 'asher hong', 'fred richard']
Total Score: 257.34200000000004
Best Seed: 88


hh

Best Selection: ['javier alfonso' 'vitaliy guimaraes' 'fred richard' 'yul moldauer']
Best Score: 254.318

In [11]:
df = df_max_scores.copy()

In [12]:
import pandas as pd
import numpy as np


# Define the number of athletes to select
num_athletes_to_select = 4

# Define the apparatuses
apparatuses = ['FX', 'PH', 'SR', 'VT', 'PB', 'HB']

# Initialize variables to store the best solution
best_score = 0
best_selection = None

# Define the number of iterations for the hyper-heuristic
num_iterations = 1000

for _ in range(num_iterations):
    # Randomly select 4 athletes
    selected_athletes = np.random.choice(df['Athlete'], size=num_athletes_to_select, replace=False)

    # Calculate the score for the current selection
    scores = df[df['Athlete'].isin(selected_athletes)][apparatuses].sum(axis=1).nlargest(3).sum()

    # Update the best solution if the current one is better
    if scores > best_score:
        best_score = scores
        best_selection = selected_athletes

# Print the best selection and score
print("Best Selection:", best_selection)
print("Best Score:", best_score)


Best Selection: ['javier alfonso' 'vitaliy guimaraes' 'fred richard' 'yul moldauer']
Best Score: 254.318


In [ ]:
ts
['asher hong' 'yul moldauer' 'khoi young' 'fred richard']
Total Score:
260.579

In [13]:
df = df_max_scores.copy()

In [14]:
import pandas as pd
import numpy as np

def calculate_score(selected_athletes, df):
    """
    Calculate the total score for a given set of selected athletes.
    """
    # Select rows corresponding to the chosen athletes
    selected_df = df[df['Athlete'].isin(selected_athletes)]

    # Calculate the sum of the top 3 scores for each apparatus
    apparatus_scores = selected_df.drop(['Athlete'], axis=1)
    top_3_sum = apparatus_scores.apply(lambda row: sum(sorted(row, reverse=True)[:3]), axis=0)

    # Calculate the total score
    total_score = top_3_sum.sum()

    return total_score




def tabu_search(df, num_athletes_to_select, tabu_size, max_iterations):
    # Initialize the solution with a random set of athletes
    available_athletes = df['Athlete'].tolist()
    current_solution = np.random.choice(available_athletes, size=num_athletes_to_select, replace=False)
    best_solution = current_solution.copy()
    best_score = calculate_score(best_solution, df)

    # Initialize tabu list
    tabu_list = []

    # Main Tabu Search loop
    for iteration in range(max_iterations):
        # Generate neighbors by swapping athletes
        neighbors = []
        for i in range(num_athletes_to_select):
            for athlete in df['Athlete']:
                if athlete not in current_solution:
                    neighbor = current_solution.copy()
                    neighbor[i] = athlete
                    neighbors.append(neighbor)

                    
                    
                best_neighbor = None
        best_neighbor_score = float('-inf')

        for neighbor in neighbors:
            neighbor_score = calculate_score(neighbor, df)

            # Convert the NumPy array to a tuple
            neighbor_tuple = tuple(neighbor)

            if neighbor_score > best_neighbor_score and neighbor_tuple not in tabu_list:
                best_neighbor = neighbor
                best_neighbor_score = neighbor_score

        # Update current solution and tabu list
        current_solution = best_neighbor
        tabu_list.append(tuple(best_neighbor))

        # Keep the tabu list size within the specified limit
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)

        # Update the best solution if a better one is found
        if best_neighbor_score > best_score:
            best_solution = best_neighbor
            best_score = best_neighbor_score

        # Print progress
        print(f"Iteration {iteration + 1}, Best Score: {best_score}")

    return best_solution, best_score



# Set parameters
num_athletes_to_select = 4
tabu_size = 10
max_iterations = 100

# Perform Tabu Search
best_solution, best_score = tabu_search(df, num_athletes_to_select, tabu_size, max_iterations)

# Print the selected athletes and the corresponding score
print('Selected Athletes:')
print(best_solution)
print('Total Score:')
print(best_score)

      

Iteration 1, Best Score: 256.84
Iteration 2, Best Score: 259.677
Iteration 3, Best Score: 260.579
Iteration 4, Best Score: 260.579
Iteration 5, Best Score: 260.579
Iteration 6, Best Score: 260.579
Iteration 7, Best Score: 260.579
Iteration 8, Best Score: 260.579
Iteration 9, Best Score: 260.579
Iteration 10, Best Score: 260.579
Iteration 11, Best Score: 260.579
Iteration 12, Best Score: 260.579
Iteration 13, Best Score: 260.579
Iteration 14, Best Score: 260.579
Iteration 15, Best Score: 260.579
Iteration 16, Best Score: 260.579
Iteration 17, Best Score: 260.579
Iteration 18, Best Score: 260.579
Iteration 19, Best Score: 260.579
Iteration 20, Best Score: 260.579
Iteration 21, Best Score: 260.579
Iteration 22, Best Score: 260.579
Iteration 23, Best Score: 260.579
Iteration 24, Best Score: 260.579
Iteration 25, Best Score: 260.579
Iteration 26, Best Score: 260.579
Iteration 27, Best Score: 260.579
Iteration 28, Best Score: 260.579
Iteration 29, Best Score: 260.579
Iteration 30, Best Score

In [15]:
data = {
    'Method': ['ts', 'hh', 'vns'],
    'Athletes': [
        ['asher hong', 'yul moldauer', 'khoi young', 'fred richard'],
        ['javier alfonso', 'vitaliy guimaraes', 'fred richard', 'yul moldauer'],
        ['fuzzy benas', 'yul moldauer', 'asher hong', 'fred richard']
    ],
    'Score': [260.579, 254.318, 257.342]
}

df = pd.DataFrame(data)

In [16]:
df

,Method,Athletes,Score
0,ts,"[asher hong, yul moldauer, khoi young, fred ri...",260.579
1,hh,"[javier alfonso, vitaliy guimaraes, fred richa...",254.318
2,vns,"[fuzzy benas, yul moldauer, asher hong, fred r...",257.342


In [18]:
# df.to_csv('usa.csv', index=False)

In [25]:
df

,Athlete,FX,PH,SR,VT,PB,HB,AA


In [28]:
df = pd.read_csv('../../../Combine_Data/men/iaa2024.csv')
df=df[df['Nation']=='USA']
df=df.drop(columns=['Rank', 'Nation', 'round', 'year'])
# Group by 'Athlete' and get the index of the maximum score for each apparatus
idx_max = df.groupby('Athlete').idxmax()

# Create a new DataFrame with the highest scores for each athlete
df_max_scores = pd.DataFrame({
    'Athlete': df.loc[idx_max['VT'], 'Athlete'].values,
    'VT': df.loc[idx_max['VT'], 'VT'].values,
    'PH': df.loc[idx_max['PH'], 'PH'].values,
    'SR': df.loc[idx_max['SR'], 'SR'].values,
    'FX': df.loc[idx_max['FX'], 'FX'].values,
    'PB': df.loc[idx_max['PB'], 'PB'].values,
    'HB': df.loc[idx_max['HB'], 'HB'].values,
    'AA': df.loc[idx_max['AA'], 'AA'].values

})

In [29]:
df

,Athlete,FX,PH,SR,VT,PB,HB,AA
421,vitaliy guimaraes,14.250,14.350,13.600,14.450,13.900,13.40,83.950
422,khoi young,13.550,15.342,12.550,15.244,13.750,13.10,83.536
423,asher hong,14.029,12.949,14.566,16.680,14.355,10.45,83.029
424,yul moldauer,13.400,13.789,14.116,13.400,15.543,11.40,81.648
425,colt walker,12.450,12.750,13.250,14.897,15.055,13.10,81.502
...,...,...,...,...,...,...,...,...
716,ian skirkey,0.000,14.972,0.000,0.000,0.000,0.00,14.972
717,ignacio yockers,0.000,14.942,0.000,0.000,0.000,0.00,14.942
718,brandon dang,0.000,14.466,0.000,0.000,0.000,0.00,14.466
719,vahe petrosyan,0.000,0.000,0.000,13.350,0.000,0.00,13.350


In [30]:
83.950+83.536+83.029

250.515

In [31]:
83.950+83.536+81.648

249.134